In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from rcrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_rcrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

In [6]:
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw]
np.sum([len(doc_words) for doc_words in docs_words])

568401

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=1, config=config)
train_docs = get_docs(instances_train, config, topic_root=topic_root)
dev_docs = get_docs(instances_dev, config, topic_root=topic_root)
test_docs = get_docs(instances_test, config, topic_root=topic_root)
init(train_docs, dev_docs, test_docs, topic_root)

## run

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train = 0.
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2] if 2 in depth_spec else 0, '%.2f'%depth_spec[3] if 3 in depth_spec else 0, \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_rcrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   182   0  0      0    525   518  0.05  0.68  0.74  0.27  0.12
1   242   1  0      0    517   506  0.05  0.68  0.75  0.42  0.20
2   285   2  0      0    508   493  0.05  0.69  0.77  0.36  0.14
3   310   3  0      0    490   478  0.05  0.66  0.71  0.32  0.18
4   343   4  0      0    464   448  0.05  0.61  0.72  0.33  0.21
5   389   5  0      0    441   422  0.05  0.60  0.74  0.34  0.19
6   409   6  0      0    418   400  0.05  0.58  0.72  0.44  0.21
7   443   7  0      0    399   383  0.05  0.58  0.74  0.45  0.19
8   473   8  0      0    385   373  0.05  0.60  0.77  0.50  0.17
9   488   9  0      0    368   356  0.05  0.60  0.77  0.46  0.15
10  487  10  0      0    354   346  0.05  0.59  0.77  0.47  0.15
11  488  11  0      0    339   334  0.05  0.59  0.76  0.44  0.15
12  524  12  0      0    326   324  0.05  0.60  0.74  0.43  0.17
13  521  13  0      0    319   317  0.05  0.60  0.74  0.42  0.17
14  521  14  0      0    312   312  0.05  0.60  0.75  0.47  0.16
15  548  15  0      0    303   308  0.05  0.60  0.78  0.41  0.14
16  544  16  0      0    300   299  0.05  0.59  0.75  0.43  0.16
17  530  17  0      0    298   297  0.05  0.59  0.79  0.46  0.13
18  519  18  0      0    293   296  0.05  0.59  0.77  0.42  0.14
19  531  19  0      0    289   289  0.05  0.59  0.76  0.38  0.16
20  535  20  0      0    285   285  0.05  0.58  0.76  0.41  0.15
21  497  21  0      0    285   285  0.05  0.58  0.81  0.52  0.12
22  534  22  0      0    281   282  0.05  0.58  0.77  0.46  0.15
23  536  23  0      0    278   280  0.05  0.59  0.77  0.50  0.14
24  531  24  0      0    277   279  0.05  0.59  0.75  0.51  0.15
25  537  25  0      0    276   279  0.05  0.59  0.76  0.54  0.15
26  539  26  0      0    275   277  0.05  0.59  0.72  0.44  0.17
27  585  27  0      0    273   275  0.05  0.60  0.75  0.45  0.17
28  598  28  0      0    270   274  0.05  0.59  0.76  0.48  0.16
29  616  29  0      0    270   274  0.05  0.60  0.77  0.45  0.14

   0 : ['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10', '0-11', '0-12', '0-13', '0-14', '0-15', '0-16', '0-17'] 753 4803.0 ['!', 'love', 'cover', 'nice', 'sleeve', ';', 'carry', '&', 'bought', 'price']
     0-1 : ['0-1-1'] 10275 56735.0 ['months', 'zipper', 'quality', 'strap', 'bought', 'broke', 'handle', 'years', "'ve", 'time']
       0-1-1 : [] 35 176.0 ['school', 'year', 'bought', 'high', 'son', 'swiss', 'months', 'road', 'ripped', 'due']
     0-2 : ['0-2-1'] 11453 67836.0 ['pocket', 'room', 'mouse', 'power', 'sleeve', 'charger', 'small', 'netbook', 'carry', 'cord']
       0-2-1 : [] 422 2146.0 ['netbook', 'inch', 'sleeve', 'acer', 'battery', 'hp', 'bought', 'aspire', 'laptops', 'made']
     0-3 : [] 15213 71645.0 ['!', 'love', 'price', 'bought', 'recommend', 'quality', 'buy', 'perfect', '...', 'color']
     0-4 : [] 13592 73323.0 ['sleeve', '&', ';', 'protection', 'pro', 'air', 'inside', 'nice', 'protect', 'padding']
     0-5 : [] 2365 16454.0 ['camera', 'car